In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [2]:
df = pd.read_csv("C:\\Users\\Lenovo\\Downloads\\new dataset\\dataset\\train.csv")

In [3]:
df.head()

,Timestamp,Residents,Apartment_Type,Temperature,Humidity,Water_Price,Period_Consumption_Index,Income_Level,Guests,Amenities,Appliance_Usage,Water_Consumption
0,01/01/2002 00,1,Studio,15.31,46.61,1.06,0.97,Low,0,Swimming Pool,0.0,64.85
1,01/01/2002 08,4,NaN,21.01,66.11,2.98,0.91,Upper Middle,1,Swimming Pool,1.0,192.50
2,01/01/2002 16,2,Cottage,12.86,60.86,1.44,1.43,Middle,0,None,1.0,116.62
3,02/01/2002 00,2,1BHK,20.16,50.58,1.48,0.91,Middle,-1,Garden,0.0,76.96
4,02/01/2002 08,2,Cottage,16.23,52.25,1.14,1.11,Middle,0,Fountain,0.0,104.70


In [4]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%d/%m/%Y %H')
df.head()

,Timestamp,Residents,Apartment_Type,Temperature,Humidity,Water_Price,Period_Consumption_Index,Income_Level,Guests,Amenities,Appliance_Usage,Water_Consumption
0,2002-01-01 00:00:00,1,Studio,15.31,46.61,1.06,0.97,Low,0,Swimming Pool,0.0,64.85
1,2002-01-01 08:00:00,4,NaN,21.01,66.11,2.98,0.91,Upper Middle,1,Swimming Pool,1.0,192.50
2,2002-01-01 16:00:00,2,Cottage,12.86,60.86,1.44,1.43,Middle,0,None,1.0,116.62
3,2002-01-02 00:00:00,2,1BHK,20.16,50.58,1.48,0.91,Middle,-1,Garden,0.0,76.96
4,2002-01-02 08:00:00,2,Cottage,16.23,52.25,1.14,1.11,Middle,0,Fountain,0.0,104.70


In [5]:
# numeric columns
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

# categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns

In [6]:
numeric_cols

Index(['Residents', 'Temperature', 'Water_Price', 'Period_Consumption_Index',
       'Guests', 'Appliance_Usage', 'Water_Consumption'],
      dtype='object')

In [7]:
categorical_cols

Index(['Apartment_Type', 'Humidity', 'Income_Level', 'Amenities'], dtype='object')

In [8]:
df.isnull().sum()

Timestamp                     0
Residents                     0
Apartment_Type              426
Temperature                 441
Humidity                      0
Water_Price                   0
Period_Consumption_Index      0
Income_Level                426
Guests                        0
Amenities                     0
Appliance_Usage             415
Water_Consumption             0
dtype: int64

In [9]:
df["Temperature"]=df["Temperature"].fillna(df["Temperature"].mean())
df["Appliance_Usage"]=df["Appliance_Usage"].fillna(df["Appliance_Usage"].mean())

In [10]:
df["Apartment_Type"]=df["Apartment_Type"].fillna('NA')
df["Income_Level"]=df["Income_Level"].fillna('NA')

In [11]:
df.isnull().sum()

Timestamp                   0
Residents                   0
Apartment_Type              0
Temperature                 0
Humidity                    0
Water_Price                 0
Period_Consumption_Index    0
Income_Level                0
Guests                      0
Amenities                   0
Appliance_Usage             0
Water_Consumption           0
dtype: int64

In [12]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

df["Apartment_Type"]=encoder.fit_transform(df["Apartment_Type"])
df["Income_Level"]=encoder.fit_transform(df["Income_Level"])
df["Amenities"]=encoder.fit_transform(df["Amenities"])
df["Humidity"]=encoder.fit_transform(df["Humidity"])


In [13]:
df.head(7)

,Timestamp,Residents,Apartment_Type,Temperature,Humidity,Water_Price,Period_Consumption_Index,Income_Level,Guests,Amenities,Appliance_Usage,Water_Consumption
0,2002-01-01 00:00:00,1,7,15.31,1476,1.06,0.97,193,0,4,0.0,64.85
1,2002-01-01 08:00:00,4,6,21.01,3404,2.98,0.91,239,1,4,1.0,192.50
2,2002-01-01 16:00:00,2,4,12.86,2901,1.44,1.43,196,0,3,1.0,116.62
3,2002-01-02 00:00:00,2,0,20.16,1874,1.48,0.91,196,-1,1,0.0,76.96
4,2002-01-02 08:00:00,2,4,16.23,2040,1.14,1.11,196,0,0,0.0,104.70
5,2002-01-02 16:00:00,4,1,22.23,2201,1.15,1.46,196,0,3,1.0,218.23
6,2002-01-03 00:00:00,3,1,10.83,2565,2.98,1.07,239,0,4,0.0,135.80


In [15]:
negative_count = (df.select_dtypes(exclude=['datetime64[ns]']) < 0).sum()
print("Negative value counts per column:")
print(negative_count)

Negative value counts per column:
Residents                   280
Apartment_Type                0
Temperature                   0
Humidity                      0
Water_Price                 272
Period_Consumption_Index      2
Income_Level                  0
Guests                      153
Amenities                     0
Appliance_Usage               0
Water_Consumption             0
dtype: int64


In [16]:
mean_value = df[df['Residents'] >= 0]['Residents'].mean()
df['Residents'] = df['Residents'].apply(lambda x: mean_value if x < 0 else x)

In [19]:
mean_value = df[df['Water_Price'] >= 0]['Water_Price'].mean()
df['Water_Price'] = df['Water_Price'].apply(lambda x: mean_value if x < 0 else x)

mean_value = df[df['Guests'] >= 0]['Guests'].mean()
df['Guests'] = df['Guests'].apply(lambda x: mean_value if x < 0 else x)

mean_value = df[df['Period_Consumption_Index'] >= 0]['Guests'].mean()
df['Period_Consumption_Index'] = df['Period_Consumption_Index'].apply(lambda x: mean_value if x < 0 else x)

In [20]:
negative_count = (df.select_dtypes(exclude=['datetime64[ns]']) < 0).sum()
print("Negative value counts per column:")
print(negative_count)

Negative value counts per column:
Residents                   0
Apartment_Type              0
Temperature                 0
Humidity                    0
Water_Price                 0
Period_Consumption_Index    0
Income_Level                0
Guests                      0
Amenities                   0
Appliance_Usage             0
Water_Consumption           0
dtype: int64


In [22]:
import os

save_dir="C:\\Users\\Lenovo\\Desktop\\Household_Water_consumption"

os.makedirs(save_dir, exist_ok=True)
df.to_csv(os.path.join(save_dir, 'preprocessed_train_data.csv'), index=False)

print(f"Data saved in '{save_dir}' directory.")

Data saved in 'C:\Users\Lenovo\Desktop\Household_Water_consumption' directory.
